### Choosing exercise routine
- choosing an exercise routine that burns the max number of calories. A function has been defined, based on the input of pushups and miles ran, will print the number of calories burned, time used, and if that time is greater than 10 minutes.

In [16]:
from pulp import *

In [15]:
# Initialize Class
model = LpProblem("Maximize calories burnt", LpMaximize)

# Define Decision Variables
num_pushup = LpVariable('Pushups', lowBound=0, upBound=None, cat='Integer')
num_miles_ran = LpVariable('Running', lowBound=0, upBound=None, cat='Integer')

# Define Objective Function
model += 3.0 * num_pushup + 130.0 * num_miles_ran

# Define Constraints
model += 0.2 * num_pushup + 10.0 * num_miles_ran <= 10.0
model += num_pushup >= 0
model += num_miles_ran >= 0

# Solve Model
model.solve()
print("{} pushups".format(num_pushup.varValue))
print("{} running".format(num_miles_ran.varValue))

50.0 pushups
0.0 running


In [1]:
def lp(num_pushup, num_miles_ran):
    """This function prints the number of calories burned, time used, and if that time is greater than 10 minutes"""
    obj = 3.0 * num_pushup + 130.0 * num_miles_ran
    print("Calories Burned: {0}".format(obj))
    min = 0.2 * num_pushup + 10.0 * num_miles_ran
    print("Time Used: {0}".format(min))
    print("Used too much time: {0}".format(min>10.0))

In [2]:
num_pushup = 50
num_miles_ran = 0
lp(num_pushup, num_miles_ran)

Calories Burned: 150.0
Time Used: 10.0
Used too much time: False


In [3]:
num_pushup = 0
num_miles_ran = 1
lp(num_pushup, num_miles_ran)

Calories Burned: 130.0
Time Used: 10.0
Used too much time: False


In [4]:
num_pushup = 30
num_miles_ran = 0.5
lp(num_pushup, num_miles_ran)

Calories Burned: 155.0
Time Used: 11.0
Used too much time: True


### Simple resource scheduling exercise
- In this exercise we are planning the production at a glass manufacturer. This manufacturer only produces wine and beer glasses:
- there is a maximum production capacity of 60 hours
- each batch of wine and beer glasses takes 6 and 5 hours respectively
- the warehouse has a maximum capacity of 150 rack spaces
- each batch of the wine and beer glasses takes 10 and 20 spaces respectively
- the production equipment can only make full batches, no partial batches

- Also, we only have orders for 6 batches of wine glasses. Therefore, we do not want to produce more than this. Each batch of the wine glasses earns a profit of `$ 5` and the beer `$ 4.5`
- The objective is to maximize the profit for the manufacturer.

In [7]:
from pulp import *

In [8]:
# Initialize Class
model = LpProblem("Maximize Glass Co. Profits", LpMaximize)

# Define Decision Variables
wine = LpVariable('Wine', lowBound=0, upBound=None, cat='Integer')
beer = LpVariable('Beer', lowBound=0, upBound=None, cat='Integer')

# Define Objective Function
model += 5 * wine + 4.5 * beer

# Define Constraints
model += 6 * wine + 5 * beer <= 60
model += 10 * wine + 20 * beer <= 150
model += wine <= 6

# Solve Model
model.solve()
print("Produce {} batches of wine glasses".format(wine.varValue))
print("Produce {} batches of beer glasses".format(beer.varValue))

C:\Users\E1366134\Anaconda3\lib\site-packages\pulp\pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Produce 6.0 batches of wine glasses
Produce 4.0 batches of beer glasses


##### Using `LpMaximize()` makes sense because we are trying to maximize profits. It does not make sense to set an upper bound on the number of wine or beer glasses produced: the more the better. However, setting a lower bound is important because producing negative glasses is impossible. Also, selecting the variable to be an integer makes sense because we can not make half batches. Adding the last constraint on wine based on current orders is important to add to avoid producing too many wine glasses.

#### Trying out lpSum
- making two types (premium and budget) of ice cream, using heavy cream, whole milk, and sugar. One version is a premium version containing more cream than our budget version.We are looking to find the mixture of ingredients that minimizes the total costs of ingredients.

```
Ingredient	| $/cup     |
:----------:|:---------:|
Cream	    | $1.5      |
Milk	    | $0.125    |
Sugar	    | $0.10     |
```

In [18]:
cb = LpVariable('budgetcream', lowBound=0, upBound=None, cat='Integer')
mb = LpVariable('budgetmilk', lowBound=0, upBound=None, cat='Integer')
sb = LpVariable('budgetsugar', lowBound=0, upBound=None, cat='Integer')
cp = LpVariable('premiumcream', lowBound=0, upBound=None, cat='Integer')
mp = LpVariable('premiummilk', lowBound=0, upBound=None, cat='Integer')
sp = LpVariable('premiumsugar', lowBound=0, upBound=None, cat='Integer')

prod_type = ['premium', 'budget']
ingredient = ['cream', 'milk', 'sugar']
var_dict = {('budget', 'cream'): cb,
 ('budget', 'milk'): mb,
 ('budget', 'sugar'): sb,
 ('premium', 'cream'): cp,
 ('premium', 'milk'): mp,
 ('premium', 'sugar'): sp}


# Define Objective Function
model += lpSum([1.5 * var_dict[(i, 'cream')] 
                + 0.125 * var_dict[(i, 'milk')] 
                + 0.1 * var_dict[(i, 'sugar')]
                
                # Iterate over product types
                for i in prod_type])

C:\Users\E1366134\Anaconda3\lib\site-packages\pulp\pulp.py:1537: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


##### In this situation using list comprehesion with lpSum() allows us to quickly define the objective function.

## Logistics planning problem
- We are consulting for kitchen oven manufacturer helping to plan their logistics for next month. There are two warehouse locations (New York, and Atlanta), and four regional customer locations (East, South, Midwest, West). The expected demand next month for East it is 1,800, for South it is 1,200, for the Midwest it is 1,100, and for West it is 1000. The cost for shipping each of the warehouse locations to the regional customer's is listed in the table below. Our goal is to fulfill the regional demand at the lowest price.


|Customer	|New York	|Atlanta   |
|:---------:|:---------:|:--------:|
|East	    |$211	    |$232      |
|South	    |$232	    |$212      |
|Midwest	|$240	    |$230      |
|West	    |$300	    |$280      |

In [23]:
costs = {('Atlanta', 'East'): 232,
 ('Atlanta', 'Midwest'): 230,
 ('Atlanta', 'South'): 212,
 ('Atlanta', 'West'): 280,
 ('New York', 'East'): 211,
 ('New York', 'Midwest'): 240,
 ('New York', 'South'): 232,
 ('New York', 'West'): 300}

atle = LpVariable('atle', lowBound=0, upBound=None, cat='Integer')
atlm = LpVariable('atlm', lowBound=0, upBound=None, cat='Integer')
atls = LpVariable('atls', lowBound=0, upBound=None, cat='Integer')
atlw = LpVariable('atlw', lowBound=0, upBound=None, cat='Integer')
ne = LpVariable('ne', lowBound=0, upBound=None, cat='Integer')
nm = LpVariable('nm', lowBound=0, upBound=None, cat='Integer')
ns = LpVariable('ns', lowBound=0, upBound=None, cat='Integer')
nw = LpVariable('nw', lowBound=0, upBound=None, cat='Integer')

var_dict = {('Atlanta', 'East'): atle,
 ('Atlanta', 'Midwest'): atlm,
 ('Atlanta', 'South'): atls,
 ('Atlanta', 'West'): atlw,
 ('New York', 'East'): ne,
 ('New York', 'Midwest'): nm,
 ('New York', 'South'): ns,
 ('New York', 'West'): nw}

1. Initialize the model.
2. `Decision Variables` :Create a list for the warehouse locations, the regional customers and the regional_demand respectively, making sure the demand and customer indices match, and zip the customers and regional_demand lists to build the demand dictionary.
3. `Define Objective` : Our decision variables here are the `number of shipments` for each warehouse and `customer combination`. We want to `minimize the total cost of shipping`. The total cost of shipping is the shipping costs along a route multiplied by the number of shipments along that route. Define the objective function using `lpSum()` with list comprehension, using the `var_dict` dictionary, iterating over customers.
4. Define the `constraint` using `lpSum`. Loop through the `list of customers`, and for each sum the shipments from the warehouses and set it equal to the customer demand. Remember we now have the customer demand as a dictionary with the customer name as the key.

In [25]:
from pulp import *

# Initialize Model
model = LpProblem("Minimize Transportation Costs", LpMinimize)

# Build the lists and the demand dictionary (Decision variables)
warehouse = ['New York', 'Atlanta']
customers = ['East', 'South', 'Midwest', 'West']
regional_demand = [1800, 1200, 1100, 1000]
demand = dict(zip(customers, regional_demand))

# Define Objective
model += lpSum([costs[(w, c)] * var_dict[(w, c)] 
                for c in customers for w in warehouse])


# For each customer, sum warehouse shipments and set equal to customer demand
for c in customers:
    model += lpSum([var_dict[(w, c)] for w in warehouse]) == demand[c]

##### Using `LpMinimize()` makes sense because we are trying to minimize shipping cost. Creating the different lists and dictionaries make it easy to define the objective function and constraints with lpSum().

## Logistics planning problem 2
- We are again consulting for kitchen oven manufacturer helping to plan their logistics. This time we are attempting to put together a plan for the next six months (Jan.-Jun.). There are still two warehouse locations (New York, and Atlanta), and four regional customer locations (East, South, Midwest, West). The cost for shipping for each of the warehouse locations to the regional customer's is listed in the table below. 
- Our goal is to determine the number of shipments from each warehouse to customers that provides the lowest costs.

|Customer |New York  |Atlanta  |
|:-------:|:--------:|:-------:|
|East	  |$211	     |$232     |
|South	  |$232	     |$212     |
|Midwest  |$240	     |$230     |
|West	  |$300	     |$280     |


In [ ]:
costs = {('Atlanta', 'East'): 232,
 ('Atlanta', 'Midwest'): 230,
 ('Atlanta', 'South'): 212,
 ('Atlanta', 'West'): 280,
 ('New York', 'East'): 211,
 ('New York', 'Midwest'): 240,
 ('New York', 'South'): 232,
 ('New York', 'West'): 300}

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
warehouse = ['New York', 'Atlanta']
customers = ['East', 'South', 'Midwest', 'West']


1. Define the decision variables in the model by first using list comprehension to iterate over the months, warehouse, and customers lists to create a list of keys. Use that list of keys with `LpVariable.dicts()` to define the variables needed.
2. Define the objective function by adding all the costs of shipping from a particular warehouse and customer over the six months.